In [1]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import math
from tqdm import tqdm
from time import sleep

In [14]:
catalog_url = 'https://www.biblio-globus.ru/catalog/categories'
catalog_page = requests.get(catalog_url)
catalog_soup = BeautifulSoup(catalog_page.content, 'lxml')
category_links = catalog_soup.find_all('li', class_='list-group-item')[:12] #Для парсинга каждой категории по отдельности тут меняем ренж [12:24] и т.д. Одновременно спокойно парсятся до 15 разделов (проверено на практике)

In [15]:
category_links

[<li class="list-group-item" style="text-align:left;"><a href="/catalog/index/182" style="font-size:.85rem; color:#333;">Классическая и современная проза</a><!-- &nbsp;<span class="badge badge-secondary">4</span> --> </li>,
 <li class="list-group-item" style="text-align:left;"><a href="/catalog/index/185" style="font-size:.85rem; color:#333;">Детективы</a><!-- &nbsp;<span class="badge badge-secondary">4</span> --> </li>,
 <li class="list-group-item" style="text-align:left;"><a href="/catalog/index/186" style="font-size:.85rem; color:#333;">Фантастика. Фэнтези</a><!-- &nbsp;<span class="badge badge-secondary">4</span> --> </li>,
 <li class="list-group-item" style="text-align:left;"><a href="/catalog/index/189" style="font-size:.85rem; color:#333;">Любовные романы. Книги о любви </a><!-- &nbsp;<span class="badge badge-secondary">4</span> --> </li>,
 <li class="list-group-item" style="text-align:left;"><a href="/catalog/index/6172" style="font-size:.85rem; color:#333;">Поэзия. Драматургия

In [25]:
df = pd.DataFrame(columns=['page_url', 'image_url', 'author', 'title', 'annotation', 'genre'])

In [ ]:
pages = 100
book_count = 1000
save_interval = 1000

for link in tqdm(category_links):
    sleep(.5)
    category_url = 'https://www.biblio-globus.ru' + link.find('a')['href']
    category_page = requests.get(category_url)
    category_soup = BeautifulSoup(category_page.content, 'lxml')
    
    subcategory_links = category_soup.find_all('a', class_='product-preview-title')

    for sublink in tqdm(subcategory_links):
        subcategory_href = sublink['href']
        subcategory_id = subcategory_href.split('/catalog/index/')[1]

        for page_count in tqdm(range(1, math.ceil(pages / len(subcategory_links)) + 1)):
            subcategory_url = f'https://www.biblio-globus.ru/catalog/category?id={subcategory_id}&page={page_count}'
            subcategory_page = requests.get(subcategory_url)
            subcategory_soup = BeautifulSoup(subcategory_page.content, 'lxml')
            product_links = subcategory_soup.find_all('a', class_='img_link')

            for product in product_links:
                product_url = 'https://www.biblio-globus.ru' + product['href']
                product_page = requests.get(product_url)
                product_soup = BeautifulSoup(product_page.content, 'lxml')

                annotation_div = product_soup.find('div', id='collapseExample')
                if annotation_div:
                    annot = ''.join([symbol for symbol in annotation_div.text if symbol not in ['\n', '\r', '\t', 'm', '\xa0']])
                    annot = annot.split('Характеристики', 1)[0]
                    annotation = annot.strip()
                else:
                    annotation = None

                meta_author = product_soup.find('meta', property='og:book:author')
                author = meta_author['content'] if meta_author else None
                
                try:
                    script_data = product_soup.find('script', type='application/ld+json').text
                    json_data = json.loads(script_data)
                except (AttributeError, json.JSONDecodeError):
                    continue

                page_url = json_data.get('offers', {}).get('url')
                image_url = json_data.get('image')
                title = json_data.get('name')
                genre = json_data.get('genre')

                temp_df = pd.DataFrame({
                    'page_url': [page_url],
                    'image_url': [image_url],
                    'author': [author],
                    'title': [title],
                    'annotation': [annotation],
                    'genre': [genre]
                })

                df = pd.concat([df, temp_df], ignore_index=True)

                book_count += 1
                if book_count % save_interval == 0:
                    df.to_csv(f'books_{book_count}.csv', index=False)
df.to_csv(f'books_all.csv', index=False)

In [24]:
df

,page_url,image_url,author,title,annotation,genre
0,https://www.biblio-globus.ru/product/10909777,https://static1.bgshop.ru/imagehandler.ashx?wi...,Сенкевич Г.,Крестоносцы,"""Крестоносцы"" — одно из лучших произведений по...",Классическая литература
1,https://www.biblio-globus.ru/product/10908601,https://static1.bgshop.ru/imagehandler.ashx?wi...,Достоевский Ф.М.,Игрок. Кроткая,Роман «Игрок» — не просто одно из всемирно изв...,Классическая литература
2,https://www.biblio-globus.ru/product/10908602,https://static1.bgshop.ru/imagehandler.ashx?wi...,Грибоедов А.С.,Горе от ума,"«„Горе от ума“ появилось раньше Онегина, Печор...",Классическая литература
3,https://www.biblio-globus.ru/product/10907353,https://static1.bgshop.ru/imagehandler.ashx?wi...,Баянов Н.,Детский гусляр,В нашем новом проекте вы найдете как книги из ...,Классическая литература
4,https://www.biblio-globus.ru/product/10907360,https://static1.bgshop.ru/imagehandler.ashx?wi...,Гоголь Н.В.,Мертвые души,История развития литературы – понятие куда бол...,Классическая литература
5,https://www.biblio-globus.ru/product/10907327,https://static1.bgshop.ru/imagehandler.ashx?wi...,Стивенсон Р.Л.,В южных морях: воспоминания и портреты,Литературное наследие классика приключенческой...,Классическая литература
6,https://www.biblio-globus.ru/product/10907380,https://static1.bgshop.ru/imagehandler.ashx?wi...,Мясницкий И.,Свадьба и другие рассказы,В нашем новом проекте вы найдете как книги из ...,Классическая литература
7,https://www.biblio-globus.ru/product/10907350,https://static1.bgshop.ru/imagehandler.ashx?wi...,Нелидова Л.Ф.,Девочка Лида,На рубеже XIX–XX вв. девочки и мальчики зачиты...,Классическая литература
8,https://www.biblio-globus.ru/product/10907365,https://static1.bgshop.ru/imagehandler.ashx?wi...,Хвощинская Н.,Пансионерка,Героиня повести Леленька живет в большой семье...,Классическая литература
9,https://www.biblio-globus.ru/product/10907344,https://static1.bgshop.ru/imagehandler.ashx?wi...,Макаренко А.С.,Педагогическая поэма,"""Педагогическая поэма"" - наиболее известное и ...",Классическая литература
